<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/two_by_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp/
from onesharp.interpreter.interpreter import *
from onesharp.tools.sanity import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lmoss/onesharp/ to /tmp/pip-req-build-9e7leyqa
  Running command git clone -q https://github.com/lmoss/onesharp/ /tmp/pip-req-build-9e7leyqa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [14]:
two_by_two_addone_idea = [
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1#1##1##'], ## note that the 1#1# adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp_idea = [
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1##1##1##'], ## note that the 1#1## adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp = sanity(two_by_two_addsharp_idea)

In [15]:
onesharp(two_by_two_addsharp,['1###'])

'1#1###'

In [23]:
def replace_add_instruction(x):
  if x == ['1#']:
    return([two_by_two_addone])
  elif x == ['1##']:
    return([two_by_two_addsharp]) 
  else:
    return(x)


In [24]:
q = replace_add_instruction(['1#'])
print(q)

['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###']


In [30]:
A = parse(diag)
B = [[x] for x in A]
C = [replace_add_instruction(c) for c in B]
C

[['1#####'],
 ['11111111111###'],
 ['111111###'],
 ['11##'],
 ['111#'],
 ['111##'],
 ['111##'],
 ['1111111####'],
 ['11#'],
 ['111#'],
 ['111##'],
 ['1111####'],
 ['111#####'],
 ['111111###'],
 ['111###'],
 ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1##1##1##1###'],
 ['1111####'],
 ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###'],
 ['11####'],
 ['11#####'],
 ['111111###'],
 ['111###'],
 ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1##1##1##1###'],
 ['1111####'],
 ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###'],
 ['11####']]